In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from astropy import units as u, constants as const
from multiprocessing import Pool
import os
os.environ["OMP_NUM_THREADS"] = "1"

from pyphot import (unit, Filter)
import bagpipes as pipes
import emcee

In [ ]:
sylt_ids = np.load('../../contrastive_learning/sylt_ids.npz')['ids']
sylt_ids

In [ ]:
band_names = ['g','r','i','z','Y','J','K','W1','W2']
eff_wavs = [4862.24, 6460.63, 7850.77, 9199.28, 9906.83, 12681.01, 21588.53, 33526, 46028]

filters_pyphot={} # For deducing photometry from spectra
for band_name, file_name in zip(band_names, [
    "CTIO_DECam.g",
    "CTIO_DECam.r",
    "CTIO_DECam.i",
    "CTIO_DECam.z",
    "CTIO_DECam.Y",
    "Paranal_VISTA.J",
    "Paranal_VISTA.Ks",
    "WISE_WISE.W1",
    "WISE_WISE.W2"
]):
    file = np.loadtxt(f'../filters/{file_name}.dat')

    wave = file[:,0]*unit['AA']
    transmit = file[:,1]
    filters_pyphot[band_name] = Filter(
        wave, transmit, name=band_name, dtype='photon', unit='Angstrom'
    )

filters_list = np.loadtxt('./filters_list_grizYJKW12.txt', dtype="str") # For bagpipes

df = pd.read_csv('../../selecting_data/objs_7102.csv',
                 index_col=0).loc[sylt_ids]
mag_df = df[[f'{band}_mag' for band in band_names]]
magerr_df = df[[f'{band}_magerr' for band in band_names]]
flux_df = df[[f'{band}_flux' for band in band_names]]
fluxerr_df = df[[f'{band}_fluxerr' for band in band_names]]
mag_df

In [ ]:
def load_grizYJKW12(ID):    
    ID=int(ID)
    
    fluxes = flux_df.loc[ID]
    fluxerrs = fluxerr_df.loc[ID] / 3 # Table has the 3sigma error; bagpipes probably wants the 1sigma
    photometry = np.vstack((flux_df.loc[ID],
                            fluxerr_df.loc[ID])).T
    return photometry

eg_phot = load_grizYJKW12(sylt_ids[1])
plt.errorbar(
    np.log10(eff_wavs),
    eg_phot[:,0],
    yerr=eg_phot[:,1],
    marker='o'
)

The next code blocks deal with producing a model galaxy, and its photometry and spectroscopy, from a few parameters, using the BAGPIPES package

In [ ]:
def package_model_components(t0,t1,mass,metallicity,dust_av,zgal):
    """
    Converts a series of galactic parameters into a model_components dictionary
     with a constant star formation rate
    """
    constant = {}  # Star formation - tophat function
    constant["age_max"] = t0 # Time since SF switched on: Gyr
    constant["age_min"] = t1 # Time since SF switched off: Gyr; t1<t0
    constant["massformed"] = mass # vary log_10(M*/M_solar) between 1 and 15
    constant["metallicity"] = metallicity # vary Z between 0 and 2.5 Z_oldsolar

    dust = {}  # Dust component
    dust["type"] = "Calzetti"  # Define the shape of the attenuation curve
    dust["Av"] = dust_av  # magnitudes
    
    nebular = {} # Nebular emission component
    nebular["logU"] = -3

    model_components = {}  # The model components dictionary
    model_components["redshift"] = zgal  # Observed redshift
    model_components["constant"] = constant
    model_components["dust"] = dust
    model_components["nebular"] = nebular
    
    return model_components

model_components = package_model_components(1,.5,10,0.2,0.2,.5) # Require t1<t0
bagpipes_galaxy_model = pipes.model_galaxy(
    model_components,
    filt_list=filters_list,
    spec_wavs=np.arange(4000., 60000., 5.)
)

def galaxy_BAGPIPES_spectroscopy(t0,t1,mass,metallicity,dust_av,zgal):
    """
    Generates a galaxy spectrum, based on a range of parameters.
    This assumes a constant star formation rate. Require t1<t0
    Outputs are in AA, Jy
    """
    model_components = package_model_components(t0, t1, mass, metallicity, dust_av, zgal)
    bagpipes_galaxy_model.update(model_components)
    wavs = bagpipes_galaxy_model.wavelengths # Rest frame
    flxs = bagpipes_galaxy_model.spectrum_full # ergscma
    
    wavs=wavs*u.AA*(1+zgal) # Redshifting
    flxs=flxs*(u.erg/u.s/(u.cm**2)/u.AA)*(wavs**2)/const.c # Converting F_lambda to F_nu
    flxs=flxs.to(u.Jy).value # Jy
    wavs=wavs.value # AA
    return wavs, flxs

def spectrum_to_photometry(wavelengths, fluxes):
    """
    Converts a spectrum to a photometry in grizYJKW12, using pyphot.
    Requires wavelengths to be in angstrom and fluxes to be in jansky
    Returns a 9d vector of the magnitudes in each band, in muJy
    """
    wavelengths*=unit['AA']
    fluxes*=unit['Jy']
    
    band_flxs = np.zeros(len(filters_pyphot))
    for i, band_name in enumerate(band_names):
        band_flxs[i] = filters_pyphot[band_name].get_flux(wavelengths, fluxes) # Jy

    return band_flxs*1e6 # muJy

wavs, flxs = galaxy_BAGPIPES_spectroscopy(1,.5,10,0.2,0.2,.6)
plt.plot(np.log10(wavs), flxs*1e6, c='r')
band_flxs = spectrum_to_photometry(wavs, flxs)
plt.plot(np.log10(eff_wavs), band_flxs, c='r',
        marker='o', mec='k')
plt.xlim(np.log10([4e3,6e4]))
plt.ylim([0,20]);

We now deal with creating a model quasar:

In [ ]:
modeldir_dict = {
        'vdb': './quasar_templates/vandenberk2001_z=0_fnu_noscale.txt',
        'sls': './quasar_templates/selsing2016_z=0_fnu_noscale.txt',
        'cln': './quasar_templates/Colina_Quasar_z=0_ETC-Glikman+Hernan-Caballero.txt',
        'mcg': './quasar_templates/mcgreer_z75qso.dat'
    }
model_qso = np.loadtxt(modeldir_dict['vdb'], skiprows=1)

filter_m_1450_file = np.loadtxt('../filters/filter_1450.txt')
def get_1450_filter(z_QSO):
    """
    Retrieves a pyphot filter which is a tophat function around 1450AA
    """
    wave = filter_m_1450_file[:,0] * unit['AA'] *(1+z_QSO)
    transmit = filter_m_1450_file[:,1]
    filter_m_1450 = Filter(wave, transmit, name='1450_tophat', dtype='photon', unit='Angstrom')
    return filter_m_1450

def quasar_spectroscopy(M_QSO, z_QSO):
    """
    Generates a quasar spectrum from a 1450A magnitude and a redshift.
    This will be based on the model chosen (default: vdb)
    Outputs are in AA, Jy
    """
    filter_m_1450 = get_1450_filter(z_QSO)
    # load quasar model, truncate Lyman-alpha forest
    spec_qso = model_qso[:,1]
    spec_qso[model_qso[:,0]*1e4<1215.16] = 0.0
    flux_qso = spec_qso *1e-3 * unit['Jy']                 # Models are apparently given in mJy...
    wavelength = model_qso[:,0]*1e4 *(1+z_QSO) * unit['AA']# ...and wavelengths in microns

    # rescale to desired apparent magnitude 1450 AA
    mag_1450 = -2.5*np.log10(filter_m_1450.get_flux(wavelength,flux_qso)/3631)
    flux_qso *= 10**( (M_QSO - mag_1450 ) / -2.5)
    return wavelength.value, flux_qso.value # in Jy

wavs, flxs = quasar_spectroscopy(20,6)
plt.plot(np.log10(wavs), flxs*1e6, c='r')
band_flxs = spectrum_to_photometry(wavs, flxs)
plt.plot(np.log10(eff_wavs), band_flxs, c='r',
        marker='o', mec='k')

Now, we set up the MCMC fitting program.

In [ ]:
def log_prior(theta, obj_type):
    """
    A uniform log prior with boundaries in a realistic range.
    Returns 0 if inside the good range
    Returns -np.inf if outside
    """
    ###### Define the log prior as uniform with boundaries in a realistic range
    if obj_type == 'GQ':
        t0,t1, mass, metallicity, dust_av, zgal, M_QSO, z_QSO = theta
        if (0<t0<13)&(0<t1<t0)&(8<mass<15)&(0<metallicity<2)&(0<dust_av<2)&(0<zgal<4)&(18<M_QSO<24)&(5.5<z_QSO<7):
            return 0.
        
    elif obj_type == 'Q':
        M_QSO, z_QSO = theta
        if (18<M_QSO<24)&(5.5<z_QSO<7):
            return 0.
        
    elif obj_type == 'G':
        t0,t1, mass, metallicity, dust_av, zgal = theta
        if (0<t0<13)&(0<t1<t0)&(8<mass<15)&(0<metallicity<2)&(0<dust_av<2)&(0<zgal<10):
            return 0.
    else:
        raise ValueError(f'Invalid obj_type: {obj_type}. Must be either `G`, `Q`, or `GQ`.')

    return -np.inf

def log_likelihood(theta, y, yerr, obj_type):
    fluxes_model = np.zeros(y.shape) # 9d vector
    
    if obj_type == 'GQ':
        t0,t1, mass, metallicity, dust_av, zgal, M_QSO, z_QSO = theta
    elif obj_type == 'Q':
        M_QSO, z_QSO = theta
    elif obj_type == 'G':
        t0,t1, mass, metallicity, dust_av, zgal = theta
    else:
        raise ValueError(f'Invalid obj_type: {obj_type}. Must be either `G`, `Q`, or `GQ`.')
    
    if 'G' in obj_type:
        wavs, flxs = galaxy_BAGPIPES_spectroscopy(t0,t1,mass, metallicity, dust_av, zgal)
        fluxes_model_galaxy = spectrum_to_photometry(wavs, flxs)
        if np.sum(fluxes_model_galaxy)==0.: # Invalid galaxy params => BAGPIPES gives a blank spectrum
            return -np.inf
        else:
            fluxes_model += fluxes_model_galaxy
        
    if 'Q' in obj_type:
        wavs, flxs = quasar_spectroscopy(M_QSO,z_QSO)
        fluxes_model_quasar = spectrum_to_photometry(wavs, flxs)
        fluxes_model += fluxes_model_quasar

    sigma2 = yerr**2
    return -0.5 * np.sum((y - fluxes_model) ** 2 / sigma2 + np.log(sigma2))

def log_probability(theta, y, yerr, obj_type):
    """
    Bayesian probability update
    """
    lp = log_prior(theta, obj_type)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, y, yerr, obj_type)

In [ ]:
def suggest_init(obj_type):
    """
    Generates an initial set of parameters, randomly distributed around mean_init_row
    """
    mean_init_row = np.array([1,.5,10,0.2,0.2,.5,20,6.])
    if obj_type=='G':
        mean_init_row = mean_init_row[:6]
    elif obj_type=='Q':
        mean_init_row = mean_init_row[6:]
    elif obj_type!='GQ':
        raise ValueError(f'Invalid obj_type: {obj_type}. Must be either `G`, `Q`, or `GQ`.')
    ndim=len(mean_init_row)
    return mean_init_row + np.random.uniform(low=-.01,high=.01,
                                             size=ndim)

def initialise_chains(nwalkers, flux, err_flux, obj_type):
    """
    Returns a set of initial vectors in parameter space, all of which within the prior.
    This is a np array of shape (nwalkers, nparams)
    """
    base_init = suggest_init(obj_type)
    pos = np.tile(base_init, (nwalkers,1))
    i=0
    while np.all(pos[-1,:]==base_init) and i<nwalkers:
        new_row = suggest_init(obj_type)
    
        # Ensure that suggested initial point is actually in the prior
        if (log_prior(new_row, obj_type)!=-np.inf) & (log_likelihood(new_row, flux, err_flux, obj_type)!=-np.inf): 
            pos[i,:] = new_row
            i+=1
    return pos

def fit_sed(flux, err_flux, obj_type,
            nsteps=5000, discard=1000, thin=10, nwalkers=32,
           **kwargs): # kwargs to be passed to EnsembleSampler
    """
    Do the MCMC sampling process to SED-fit the fluxes (and their errors) to a G, Q, or GQ model
    Returns each of the samples, and their log_probs, in a big array.
    """
    
    init = initialise_chains(nwalkers, flux, err_flux, obj_type)
    nwalkers, ndim = init.shape
    
    with Pool() as pool: # Multicore processing
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability,
                                        args=(flux, err_flux, obj_type),
                                        pool=pool,
                                       **kwargs)
        sampler.run_mcmc(init, nsteps, progress=True)

    samples = sampler.get_chain(flat=False,thin=thin,discard=discard)
    log_prob = sampler.get_log_prob(flat=False,thin=thin,discard=discard)

    return samples, log_prob

The following will run an MCMC fit for a galaxy+quasar model on the data for the first object. It takes about 7 mins with the default parameters to the `fit_sed` function.

In [ ]:
photometry = load_grizYJKW12(sylt_ids[0])
fluxes_muJy = photometry[:,0]
err_fluxes_muJy = photometry[:,1]

samples, log_prob = fit_sed(flux=fluxes_muJy, err_flux=err_fluxes_muJy, obj_type='GQ',
#                             nsteps=10000, discard=4000, thin=2,
                            nwalkers=32, a=2.
                                ) # ~7min

---
We now begin to analyse the results of the MCMC fitting of all 12 objects to all 3 models. This is done by the file `quasar_galaxy_fit.py` (not `.ipynb`) and are stored in `./mcmc_results.npz`

In [ ]:
MODELS=['G','Q','GQ']
mcmc_results = np.load('./mcmc_results.npz')

ids = mcmc_results['ids']
samples_results = {}
samples_results['G'] = mcmc_results['samples_G']
samples_results['Q'] = mcmc_results['samples_Q']
samples_results['GQ'] = mcmc_results['samples_GQ']
log_probs_results = {}
log_probs_results['G'] = mcmc_results['log_probs_G']
log_probs_results['Q'] = mcmc_results['log_probs_Q']
log_probs_results['GQ'] = mcmc_results['log_probs_GQ']

print(ids.shape)
print([samples_results[model].shape for model in MODELS])
print([log_probs_results[model].shape for model in MODELS])

In [ ]:
def best_params(samples, log_prob):
    """
    Calculates the set of parameters with the highest probability
    Returns the highest prob set of parameters, and the log of that probability
    """
    nparams = samples.shape[2]
    flat_samples = samples.reshape(
        (samples.shape[0]*samples.shape[1],nparams)
    )
    pred_params = flat_samples[np.argmax(log_prob)]
    max_log_prob = np.max(log_prob)
    return pred_params, max_log_prob

def show_convergence(samples, log_prob, obj_type, burn=0):
    """
    Visualises the sampling over time, and the posterior distributions
    """
    samples = samples[burn:]
    log_prob = log_prob[burn:]
    nparams = samples.shape[2]
    fg, axs = plt.subplots(nparams,2,figsize=(11,4+3*nparams), sharey='row', sharex='col',
                           gridspec_kw={'width_ratios':[2,1]})
    pred_params, _ = best_params(samples, log_prob)
    params = ["t0","t1","mass", "metallicity", "dust_av", "z_G", "M_QSO", 'z_QSO']
    if obj_type=='G':
        params = params[:6]
    elif obj_type=='Q':
        params = params[6:]
    
    flat_samples = samples.reshape(
        (samples.shape[0]*samples.shape[1],nparams)
    )
    
    for i in range(nparams):
        param_samples = samples[:,:,i].reshape(samples.shape[0]*samples.shape[1])
        
        x = np.tile(np.arange(samples.shape[0]), (samples.shape[1],1)).T.reshape(samples.shape[0]*samples.shape[1])
        axs[i,0].scatter(x, param_samples,
                         s=.01, c=np.arange(param_samples.shape[0])%samples.shape[1],
                         cmap='rainbow')
        axs[i,0].set_ylabel(params[i])
        axs[i,1].hist(flat_samples[:,i], orientation='horizontal', bins=50)
        axs[i,1].axhline(pred_params[i], c='r')
    axs[0,0].set_title('Values of parameters')
    axs[0,1].set_title('Histogram')
    axs[nparams-1,0].set_xlabel('Step')
    fg.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0.1)
    return fg

fg = show_convergence(
    samples_results['Q'][ids==ids[10],:,:,:][0],
    log_probs_results['Q'][ids==ids[10],:,:][0],
    'Q'
)

In [ ]:
best_params_dict = {}
nparams = {
    'G':6,
    'Q':2,
    'GQ':8
}
for model in MODELS:
    best_params_model = np.zeros((len(ids), nparams[model]))
    for idd in ids:
        samples = samples_results[model][ids==idd,:,:,:][0]
        log_probs = log_probs_results[model][ids==idd,:,:][0]
        best_params_model[ids==idd,:] = best_params(samples, log_probs)
    best_params_dict[model] = best_params_model

In [ ]:
nparams_inv = {v: k for k, v in nparams.items()}

wavs = np.logspace(np.log10(4e3),np.log10(6e4), num=10000)
def spectrum_from_params(params):
    """
    Retrieves the spectrum of a model from its parameter list
    """
    ot = nparams_inv[len(params)]
    fluxes = np.zeros_like(wavs)
    if 'G' in ot:
        wvs, fxs = galaxy_BAGPIPES_spectroscopy(
            params[0], params[1], params[2], params[3], params[4], params[5]
        )
        fluxes += np.interp(wavs, wvs, fxs)
    if 'Q' in ot:
        wvs, fxs = quasar_spectroscopy(
            params[-2], params[-1]
        )
        fluxes += np.interp(wavs, wvs, fxs)
    return fluxes * 1e6 # muJy

plt.plot(np.log10(wavs), 
         spectrum_from_params(
             best_params_dict['GQ'][0,:]
         )
        )

In [ ]:
def BICs(idd):
    """
    Returns the BICs for G, Q, GQ models for a given ID
    BIC = -2lnL + JlnN
    """
    bics = np.zeros(3)
    for i, model in enumerate(MODELS):
        _, lnL = best_params(
            samples_results[model][ids==idd,:,:][0],
            log_probs_results[model][ids==idd,:][0]
        )
        bics[i] = -2*lnL + nparams[model]*np.log(9)
    return bics
bics = np.zeros((len(ids),3))
for i, idd in enumerate(ids):
    bics[i,:] = BICs(idd)
bics

In [ ]:
idd=ids[11]

best_obj_params = [best_params_dict[model][ids==idd,:][0] for model in MODELS]
model_spectra = np.array([spectrum_from_params(param_set) for param_set in best_obj_params])

fg, ax = plt.subplots(figsize=(10,7))
ax.errorbar(
    np.log10(eff_wavs),
    [df.loc[idd][f'{band}_flux'] for band in band_names],
    yerr=[df.loc[idd][f'{band}_fluxerr'] for band in band_names],
    c='r', marker='o',capsize=7,elinewidth=2, mew=2, mec='k',# markersize=10
)

ax.plot(
    np.log10(wavs), model_spectra[0,:], 'g',
    np.log10(wavs), model_spectra[1,:], 'orange',
    np.log10(wavs), model_spectra[2,:], 'm'
)

ax.legend(
    [f'{MODELS[i]}: BIC$=${bics[ids==idd][0][i]:.1f}' \
           for i in range(3)] + ['True fluxes'],
    fontsize=16
)

model_fluxes = np.array([
    filters_pyphot[band_name].get_flux(
        wavs*unit['AA'],
        model_spectra*1e-6*unit['Jy']
    )*1e6 for band_name in band_names
]).T
ax.plot(np.log10(eff_wavs), model_fluxes[0,:], 'g',
        np.log10(eff_wavs), model_fluxes[1,:], 'orange',
        np.log10(eff_wavs), model_fluxes[2,:], 'b',
        marker='o', lw=0, mec='k')
ax.set_xlim(np.log10([4e3, 6e4]))
ax.set_ylim(np.max(model_fluxes)*np.array([-.05,1.6]))

ax.set_title(
    f'ID: {idd}',
    fontsize=24
);

fg.savefig(f'fitting_spectra/{idd}.png')